In [10]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

def save_to_json(bidder_demand_dict, filename):
    """
    Save bidder_demand_dict to a JSON file, ensuring type compatibility.

    Parameters:
    - bidder_demand_dict: dict, a dictionary with bidders as keys containing quantity and price demand
    - filename: str, the name of the output file (e.g., "bidder_demand_dict.json")
    """
    # Create a converted dictionary to ensure all data types are JSON serializable
    def convert_to_serializable(obj):
        """ Recursively convert non-JSON-serializable types to native Python types. """
        if isinstance(obj, dict):
            return {k: convert_to_serializable(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_serializable(item) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(convert_to_serializable(item) for item in obj)
        elif isinstance(obj, (int, float, str)):
            return obj
        else:
            # Attempt to convert to string as a fallback
            return str(obj)

    # Convert the dictionary to a serializable format
    serializable_dict = convert_to_serializable(bidder_demand_dict)
    
    # Write the converted data to a JSON file
    try:
        with open(filename, 'w') as f:
            json.dump(serializable_dict, f, indent=4)
        print(f"Data successfully saved to file: {filename}")
    except (TypeError, ValueError) as e:
        print(f"Failed to save file: {e}")

def load_from_json(filename):
    """
    Load from a JSON file.

    Parameters:
    - filename: str, the name of the JSON file to load (e.g., "bidder_demand_dict.json")

    Returns:
    - dict: the loaded dictionary with bidders as keys and their demand data as values
    """
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        print(f"Data successfully loaded from file: {filename}")
        return data
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Failed to load file: {e}")
        return None
    
bidder_demand_dict = load_from_json("../data/processed data/bidder_demand_dict.json")
garp_violation_results = load_from_json("../output/rp_checking/garp_violation_results.json")

Data successfully loaded from file: ../data/processed data/bidder_demand_dict.json
Data successfully loaded from file: ../output/rp_checking/garp_violation_results.json


In [5]:
import statsmodels.api as sm

def filter_bidder_data_by_garp(bidder_demand_dict, garp_violation_results):
    """
    Filter bidder_demand_dict by removing round data that violates GARP,
    based on GARP verification results.

    Parameters:
        bidder_demand_dict (dict): Original bidder demand data.
        garp_violation_results (dict): Output from GARP checking function.

    Returns:
        dict: Filtered bidder demand data with GARP-violating rounds removed.
    """
    filtered_bidder_demand_dict = {}
    
    for bidder, round_data in bidder_demand_dict.items():
        if garp_violation_results[bidder]['has_violation']:
            # Start with all round indices and remove the ones involved in GARP violations
            valid_rounds = set(range(len(round_data)))
            for violation in garp_violation_results[bidder]['pair_violation']:
                round_pair = violation['round_pair']
                if violation['violation']:
                    valid_rounds.discard(round_pair[0])
                    valid_rounds.discard(round_pair[1])
            
            # Keep only rounds not marked as GARP violations
            filtered_bidder_demand_dict[bidder] = [round_data[i] for i in valid_rounds]
        else:
            # No violation — keep all rounds
            filtered_bidder_demand_dict[bidder] = round_data
    
    return filtered_bidder_demand_dict

def estimate_bidder_parameters(bidder_demand_data):
    """
    Estimate a bidder's demand curve parameters (theta_i, sigma_i)
    and rivals' demand response (theta_r, sigma_r) using linear regression.

    Parameters:
        bidder_demand_data (list): List of tuples containing 
        (round_id, quantity_vector, price_vector, rivals_quantity_vector)

    Returns:
        tuple: (theta_i, sigma_i, theta_r, sigma_r)
    """
    prices = []
    demands = []
    rivals_demands = []
    
    for _, quantity_vector, price_vector, rivals_quantity_vector in bidder_demand_data:
        prices.extend(price_vector)
        demands.extend(quantity_vector)
        rivals_demands.extend(rivals_quantity_vector)
    
    # Add constant term to price for intercept estimation
    X = sm.add_constant(prices)

    # Bidder's demand curve estimation
    model_i = sm.OLS(demands, X)
    results_i = model_i.fit()

    # Rivals' aggregate demand response estimation
    model_r = sm.OLS(rivals_demands, X)
    results_r = model_r.fit()
    
    theta_i = results_i.params[0]  # intercept
    sigma_i = -results_i.params[1]  # negative slope

    theta_r = results_r.params[0]
    sigma_r = -results_r.params[1]
    
    return theta_i, sigma_i, theta_r, sigma_r

In [7]:
# Step 1: Remove GARP-violating rounds before estimation
filtered_bidder_demand_dict = filter_bidder_data_by_garp(bidder_demand_dict, garp_violation_results)

# Step 2: Estimate demand curve parameters for each bidder
estimated_parameters = {
    bidder: estimate_bidder_parameters(data)
    for bidder, data in filtered_bidder_demand_dict.items()
}

save_to_json(estimated_parameters, "../output/identification/estimated_parameters.json")

Data successfully saved to file: ../output/identification/estimated_parameters.json


In [11]:
def calculate_shadow_prices(bidder_demand_dict, estimated_parameters):
    """
    Calculate the shadow prices of rival utility for each round and each bidder.

    Parameters:
        bidder_demand_dict (dict): Filtered bidder demand data.
        estimated_parameters (dict): Estimated parameters per bidder: (theta_i, sigma_i, theta_r, sigma_r)

    Returns:
        dict: Nested dictionary mapping each bidder to a dictionary of round_id to shadow price list.
              Format: {bidder: {round_id: [shadow_price_market1, ...]}}
    """
    shadow_prices = defaultdict(dict)

    for bidder, rounds in bidder_demand_dict.items():
        # Load rival parameters
        try:
            _, _, theta_r, sigma_r = estimated_parameters[bidder]
        except KeyError:
            print(f"Skipping bidder {bidder} due to missing estimated parameters.")
            continue

        for round_id, _, _, rivals_quantity_vector in rounds:
            rivals_effect = []
            for q in rivals_quantity_vector:
                if q == 0:
                    rivals_effect.append(0)
                else:
                    rivals_effect.append(theta_r - sigma_r * np.log(q))
            shadow_prices[bidder][round_id] = rivals_effect

    return shadow_prices


In [12]:
shadow_prices = calculate_shadow_prices(filtered_bidder_demand_dict, estimated_parameters)
save_to_json(shadow_prices, "../output/identification/shadow_prices.json")

Data successfully saved to file: ../output/identification/shadow_prices.json


In [13]:
def identify_all_spitefulness(bidder_demand_dict, shadow_prices):
    """
    Identify spiteful bidding behavior for all bidders by estimating alpha_i,
    reflecting how much each bidder values reducing rivals' utility.

    Parameters:
        bidder_demand_dict (dict): Filtered bidder demand data.
        shadow_prices (dict): Nested dict: shadow_prices[bidder][round_id] = vector

    Returns:
        dict: A dictionary mapping each bidder to its spitefulness result:
              {bidder: {'has_spitefulness': bool, 'alpha_hat': float}}
    """
    result_dict = {}

    for bidder, round_data in bidder_demand_dict.items():
        print(f"Processing bidder: {bidder}")
        try:
            bidder_shadow_prices = shadow_prices[bidder]
        except KeyError:
            print(f"  [Warning] Missing shadow prices for {bidder}. Skipping.")
            continue

        # Parameters can be fixed or bidder-specific; here we assume fixed (can be improved)
        theta_i = 25.565
        sigma_i = 0.684

        has_spitefulness = False
        alpha_hat = 0.0
        min_violations = float('inf')

        for alpha in np.linspace(0, 1, 21):
            total_violations = 0

            for (round_id, quantity_vector, price_vector, _) in round_data:
                quantity_vector = [np.log(q) if q > 0 else 0 for q in quantity_vector]
                price_vector = [np.log(p) if p > 0 else 0 for p in price_vector]

                if round_id not in bidder_shadow_prices:
                    continue

                shadow_price = bidder_shadow_prices[round_id]

                adjusted_price_vector = [
                    (1 - alpha) * p - alpha * r
                    for p, r in zip(price_vector, shadow_price)
                ]

                for other_round_id, other_quantity_vector, other_price_vector, _ in round_data:
                    if round_id == other_round_id:
                        continue

                    expenditure_a_a = sum(p * q for p, q in zip(adjusted_price_vector, quantity_vector))
                    expenditure_a_b = sum(p * q for p, q in zip(adjusted_price_vector, other_quantity_vector))
                    expenditure_b_b = sum(p * q for p, q in zip(other_price_vector, other_quantity_vector))
                    expenditure_b_a = sum(p * q for p, q in zip(other_price_vector, quantity_vector))

                    if expenditure_a_b < expenditure_a_a and expenditure_b_a < expenditure_b_b:
                        total_violations += 1

            if total_violations < min_violations:
                min_violations = total_violations
                alpha_hat = alpha
                has_spitefulness = alpha > 0

        result_dict[bidder] = {
            'has_spitefulness': has_spitefulness,
            'alpha_hat': alpha_hat
        }

    return result_dict


In [14]:
spitefulness_results = identify_all_spitefulness(filtered_bidder_demand_dict, shadow_prices)

# 输出保存
save_to_json(spitefulness_results, "../output/identification/spitefulness_results.json")

# 示例输出打印
for bidder, result in spitefulness_results.items():
    print(f"{bidder}: Alpha_hat = {result['alpha_hat']:.2f}, Has Spitefulness = {result['has_spitefulness']}")


Processing bidder: 8538 Green Street LLC
Processing bidder: AMA Communications, LLC
Processing bidder: AT&T Spectrum Frontiers LLC
Processing bidder: Agri-Valley Communications, Inc.
Processing bidder: Bluegrass Consortium
Processing bidder: Cal.net, Inc.
Processing bidder: California Internet, L.P. dba GeoLinks
Processing bidder: Canopy Spectrum, LLC
Processing bidder: Carolina West Wireless, Inc.
Processing bidder: Cellco Partnership
Processing bidder: Cellular South Licenses, LLC
Processing bidder: Central Texas Telephone Investments, LP
Processing bidder: Cox Communications , Inc.
Processing bidder: Cross Telephone Company, L.L.C.
Processing bidder: East Kentucky Network, LLC
Processing bidder: Etheric Networks Incorporated
Processing bidder: FTC Management Group, Inc.
Processing bidder: Grand River Communications, Inc.
Processing bidder: Granite Wireless LLC
Processing bidder: Hilliary Acquisition Corp 2016, LLC
Processing bidder: Horry Telephone Cooperative, Inc.
Processing bidde